In [7]:
!pip install fuzzywuzzy
!pip install langdetect
!pip install stop_words
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import urllib.request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from fuzzywuzzy import fuzz
from string import digits
from stop_words import get_stop_words
from nltk.corpus import stopwords
from fuzzywuzzy import process

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
punct = punctuation+'©«»—…“”*№– ' + digits
stop_words_list = get_stop_words('en')
stop_words_list2 = stopwords.words('english')
newlist = stop_words_list + stop_words_list2
for word in ["свой"]:
    newlist.append(word)
newlist_set = set(newlist)

In [0]:
def which_language (text):
  import re
  if bool(re.search('[\u0400-\u04FF]', text)) == True:
    return 'russian'
  else:
    return 'english'

In [0]:
teams_names_in_russian_and_english_df = pd.read_csv("teams names in russian and english.csv")
leagues_names_in_russian_and_english_df = pd.read_csv("leagues names in russian and english.csv")

In [11]:
df = pd.read_csv('result.tar.gz', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df['result.tsv'] = df['result.tsv'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'https\S+', '', regex=True)
df = df.replace('', np.NaN)
df = df.replace('\xa0', np.NaN)
df = df.dropna()
df =  df[df['result.tsv'].str.contains('[A-Za-z]')]
df = df[df['result.tsv'].str.split().str.len().gt(1)]

In [0]:
df = df.dropna()

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.reset_index(drop=True)

In [17]:
df.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
1,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония
2,Украинская команда имеет отличный шанс выйти в...,1.55,Заря забьет 1 или 2 гола: да,1xСтавка,2019-08-15 17:00:00,Лига Европы УЕФА,Заря,ЦСКА София
3,«Сент-Миррен» в родных стенах встретится с «Аб...,2.55,«Абердин» фора (-1),BETCITY,2019-08-11 14:00:00,Премьершип,Сент-Миррен,Абердин
4,На стадионе «Айброкс» состоится встреча между ...,1.97,"«Абердин» фора (+1,5)",Марафон,2019-09-28 14:00:00,Премьершип,Глазго Рейнджерс,Абердин


In [0]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
  lang.append(which_language(str(text)))

In [0]:
df['Language'] = lang

In [0]:
grouped=df.groupby('Language')

In [0]:
for name, group in grouped: 
  if name == 'english': 
    english = group.drop('Language',axis=1) 
  else: 
    russian = group.drop('Language',axis=1)

In [0]:
english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_1_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_2_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, leagues_names_in_russian_and_english_df,  how='left', left_on=['league_name_in_russian'], right_on = ['leagues name in Russian'])
english.rename({'unique_leagues_in_english': 'league_name_in_english'}, axis=1, inplace=True)


In [0]:
english.drop(['team name in Russian_x' , 'team name in Russian_y' , 'leagues name in Russian' ] , axis=1 , inplace=True)

In [0]:
# Here we are just re-naming the columns so one check that what's in it and interpret the information easily 
english.rename({'team name in English_x': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)
english.rename({'team name in English_y': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)
english.rename({'leagues names in English': 'league_name_in_english'}, axis=1, inplace=True)

In [0]:
# Here we are just setting the correct data type that english names should be string
english['team_of_season_1_name_in_english']=english['team_of_season_1_name_in_english'].astype(str)
english['team_of_season_2_name_in_english']=english['team_of_season_2_name_in_english'].astype(str)
english['league_name_in_english']=english['league_name_in_english'].astype(str)

In [26]:
english.head(2)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League


In [27]:
unique_leagues = english['league_name_in_russian'].unique().tolist()
len(unique_leagues)

86

In [28]:
unique_teams1 = english['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

1261

In [29]:
unique_teams2 = english['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

1290

In [30]:
unique_teams = unique_teams1 + unique_teams2
unique_teams_set = set(unique_teams)
len(unique_teams_set)

1400

In [0]:
english = pd.read_csv('english_custom_lists_and_directmatch.csv')

In [0]:
english = english[english['result.tsv'].map(len) > 30]

In [0]:
english = english.reset_index(drop=True)

In [34]:
english.shape

(87756, 13)

In [35]:
english.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Middlesbrough have woken to good times in thei...,7.0,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship,0.0,0.0
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League,0.0,0.0
2,In my opinion a 0-0 draw is quite likely betwe...,13.0,0-0,bet365,2019-04-18 19:00:00,Лига Европы УЕФА,Валенсия,Вильярреал,Valencia,Villarreal,UEFA Europa League,0.0,0.0


In [36]:
subset_english = english.sample(n=10000)
subset_english.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
55698,Morecambe scored twice in last home match and ...,3.16,W1,bet365,2019-09-07 14:00:00,Вторая Лига,Моркам,Салфорд Сити,Morecambe,Salford City,League Two,0.0,0.0
74744,Napoli made a great start to their Champions L...,1.44,W2,bet365,2019-10-02 16:55:00,Лига чемпионов УЕФА,Генк,Наполи,Genk,Napoli,UEFA Champions League,0.0,0.0
4588,Fantastic game between two very attacking team...,22.00,2-3,bet365,2019-04-28 19:30:00,МЛС,Сиэтл Саундерз,ФК Лос-Анджелес,Seattle Saunders,FC Los Angeles,MLS,0.0,0.0
67987,Mjondalen have won two and drawn two recently ...,2.90,W2,bet365,2019-06-30 16:00:00,Элитсерия,ФК Мьёндален,ФК Викинг,FC mjøndalen if,Viking FK,Elitserien,0.0,0.0
15142,I. Perisic - two scored goals in last four mat...,6.00,First Goalscorer: Ivan Perisic,bet365,2019-09-09 16:00:00,Чемпионат Европы. Квалификация,Азербайджан,Хорватия,Azerbaijan,Croatia,Europe championship. Qualification,0.0,0.0


In [37]:
subset_english = subset_english.reset_index(drop=True)
subset_english.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Morecambe scored twice in last home match and ...,3.16,W1,bet365,2019-09-07 14:00:00,Вторая Лига,Моркам,Салфорд Сити,Morecambe,Salford City,League Two,0.0,0.0
1,Napoli made a great start to their Champions L...,1.44,W2,bet365,2019-10-02 16:55:00,Лига чемпионов УЕФА,Генк,Наполи,Genk,Napoli,UEFA Champions League,0.0,0.0
2,Fantastic game between two very attacking team...,22.00,2-3,bet365,2019-04-28 19:30:00,МЛС,Сиэтл Саундерз,ФК Лос-Анджелес,Seattle Saunders,FC Los Angeles,MLS,0.0,0.0
3,Mjondalen have won two and drawn two recently ...,2.90,W2,bet365,2019-06-30 16:00:00,Элитсерия,ФК Мьёндален,ФК Викинг,FC mjøndalen if,Viking FK,Elitserien,0.0,0.0
4,I. Perisic - two scored goals in last four mat...,6.00,First Goalscorer: Ivan Perisic,bet365,2019-09-09 16:00:00,Чемпионат Европы. Квалификация,Азербайджан,Хорватия,Azerbaijan,Croatia,Europe championship. Qualification,0.0,0.0


Team 1:

In [38]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_en = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85:
                    #print(english['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams1'][i] = 1

                    break

137it [00:15, 11.47it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10000it [21:03,  7.92it/s]


In [39]:
success = subset_english[subset_english['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

58
10000
0.0058


In [40]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
#         print(i, good_words)
#         print(subset_english['team_of_season_1_name_in_russian'][i])
        
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 65 and fuzz.ratio(word, subset_english['team_of_season_1_name_in_russian'][i]) > 65:
#                     print(subset_english['team_of_season_1_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams1'][i] = 1

                    break

10000it [31:44,  5.25it/s]


In [41]:
success = subset_english[subset_english['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

58
10000
0.0058


Team2:

In [42]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85 and subset_russian['team_of_season_2_name_in_russian'][i] == word:
                    #print(english['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams2'][i] = 1

                    break

10000it [31:58,  5.21it/s]


In [43]:
success = subset_english[subset_english['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

3
10000
0.0003


In [44]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    if subset_english['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([A-Z][a-z]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_en + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 65 and fuzz.ratio(word, subset_english['team_of_season_2_name_in_russian'][i]) > 65:                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_english['list_direct_match_for_teams2'][i] = 1

                    break

10000it [31:39,  5.27it/s]


In [45]:
success = subset_english[subset_english['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_english['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

3
10000
0.0003


In [0]:
subset_english.to_csv('fuzzylistsnew20106.csv', index = False)

League:

In [0]:
leagues_match = np.zeros(subset_english.shape[0])

In [48]:
for i, text in tqdm(enumerate(subset_english['result.tsv'].values)):
    #print(i)
    capital_words_ru = re.findall(r'([A-Z][a-z]+)', text)
    quotes = re.findall(r'\"(.+?)\"', text)
    quotes2 = re.findall(r'«(.+?)»', text)
    quotes3 = re.findall(r'\'(.+?)\'', text)
    search_words = capital_words_en + quotes + quotes2 + quotes3
    search_words = set(search_words)
    good_words = [word for word in search_words if word.lower() not in newlist_set] 
    #print(good_words)
    
    for word in good_words:
        for league in unique_leagues:
            if fuzz.ratio(word, league) > 50 and fuzz.ratio(word, subset_english['league_name_in_russian'][i]) > 50:
                print(subset_english['league_name_in_russian'][i], word, league, fuzz.ratio(word, league))
                #print('yes')
                leagues_match[i] = 1

                break

10000it [01:13, 136.67it/s]


In [0]:
subset_english['list_fuzzy_match_for_league'] = leagues_match

In [50]:
success = subset_english[subset_english['list_fuzzy_match_for_league'] == 1].shape[0]
print(success)
all_vals = subset_english['list_fuzzy_match_for_league'].shape[0]
print(all_vals)
baseline_league = success / all_vals
print(baseline_league)

0
10000
0.0


In [0]:
subset_english.to_csv('fuzzylists0306_withleagues.csv', index = False)